In [58]:
import tensorflow as tf
#import tensorflow.contrib.eager as tfe
import os
import matplotlib.pyplot as plt
import re
import numpy as np
from string import punctuation
from collections import defaultdict
from functools import reduce
from keras.preprocessing.sequence import pad_sequences
from itertools import chain
from InputPreparator import EmbeddingsPreparator
from InputPreparator import StoryParser
import time

#to avoid a warning from TF 1.7 version see https://github.com/tensorflow/tensorflow/issues/18111
import warnings
warnings.filterwarnings('ignore')


# Input preparation

In [269]:
def get_task_files(task_nr):
    if task_nr==5:
        return 'qa5_three-arg-relations_train.txt', "qa5_three-arg-relations_test.txt"
    if task_nr==6:
        return 'qa6_yes-no-questions_train.txt', 'qa6_yes-no-questions_test.txt'
    if task_nr==10:
        return 'qa10_indefinite-knowledge_train.txt', 'qa10_indefinite-knowledge_test.txt'

In [270]:
train_set_file = get_task_files(6)[0]
test_set_file = get_task_files(6)[1]

train_set_post_file = "data/tasks_1-20_v1-2/en/"+train_set_file
test_set_post_file = "data/tasks_1-20_v1-2/en/"+test_set_file

In [271]:
embedder=EmbeddingsPreparator()
story_parser=StoryParser()

In [272]:
vocab_tokens = embedder.get_unique_tokens([train_set_post_file, test_set_post_file])

In [273]:
word_to_index, index_to_embedding = embedder.load_embedding_from_disks("data/glove.6B.50d.txt",vocab_tokens, with_indexes=True)

In [274]:
train_stories=story_parser.get_stories(train_set_post_file, True)
test_stories=story_parser.get_stories(test_set_post_file, True)

In [275]:
contexts_train, questions_train, answers_train = story_parser.vectorize_stories(train_stories, word_to_index)
contexts_test, questions_test, answers_test = story_parser.vectorize_stories(test_stories, word_to_index)

In [276]:
print('contexts.shape = {}'.format(contexts_train.shape))
print('questions.shape = {}'.format(questions_train.shape))
print('answers.shape = {}'.format(answers_train.shape))

contexts.shape = (1000,)
questions.shape = (1000, 6)
answers.shape = (1000, 38)


## Embedding Layer

In [277]:
embedding_matrix = np.zeros((len(word_to_index) + 1, embed_dimensions))
for word, i in word_to_index.items():
    embedding_vector = index_to_embedding[i]
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [278]:
final_train_data = story_parser.get_final_dataset(contexts_train, questions_train, answers_train)
final_test_data = story_parser.get_final_dataset(contexts_test, questions_test, answers_test)

In [279]:
def prep_data(data, all_data= False, train= False):
    contextsvs, questionsvs, answers=zip(*data)
    
    #Pad to longest sequence in the batch 
    contexts = list(contextsvs)
    max_context_length = max([len(x) for x in contexts])
    questions = list(questionsvs)
    max_query_length = max(len(x) for x in questionsvs)

    final_contexts=pad_sequences(contextsvs, maxlen=max_context_length) 
    final_queries=pad_sequences(questionsvs, maxlen=max_query_length)
    
    return final_contexts, final_queries

# Model

In [280]:
final_contexts, final_queries= prep_data(final_train_data)

vocab_size= len(index_to_embedding)
embed_dimensions= 50
num_units_gru= 50
keep_prob= 0.5
#q_shape= final_queries.shape[1]
#c_shape= final_contexts.shape[1]

In [281]:
q_in = tf.keras.Input(shape=(None,))
c_in = tf.keras.Input(shape=(None,))

#embed
embed = tf.keras.layers.Embedding(len(word_to_index) + 1,
                            embed_dimensions,
                            weights=[embedding_matrix],
                            trainable=False)
q = embed(q_in)
c = embed(c_in)

#encode
grucell= tf.keras.layers.GRUCell(num_units_gru)
c = tf.keras.layers.RNN(grucell)(c)
q = tf.keras.layers.RNN(grucell)(q)

#add dropout
dropout= tf.keras.layers.Dropout(keep_prob)
encoded_sentence= dropout(c)
encoded_question= dropout(q)

#merge
merged= tf.keras.layers.concatenate([encoded_sentence, encoded_question])

#predict
pred=tf.keras.layers.Dense(answers_train.shape[1], activation=tf.nn.softmax)(merged)

model = tf.keras.Model(inputs=[q_in, c_in], outputs=pred)

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_36 (InputLayer)           (None, None)         0                                            
__________________________________________________________________________________________________
input_35 (InputLayer)           (None, None)         0                                            
__________________________________________________________________________________________________
embedding_17 (Embedding)        (None, None, 50)     1900        input_35[0][0]                   
                                                                 input_36[0][0]                   
__________________________________________________________________________________________________
rnn_33 (RNN)                    (None, 50)           15150       embedding_17[1][0]               
__________

## Train

In [282]:
print('Training...')
start_time = time.time()
model.fit([final_queries, final_contexts], answers_train, epochs=200, batch_size=128)
elapsed_time = time.time() - start_time
print()
print('Training time: ')
print(elapsed_time)

Training...
Epoch 1/200
1000/1000 [==============================] - 7s 7ms/step - loss: 3.3455 - acc: 0.1330
Epoch 2/200
1000/1000 [==============================] - 0s 319us/step - loss: 1.9578 - acc: 0.5140
Epoch 3/200
1000/1000 [==============================] - 0s 319us/step - loss: 1.1881 - acc: 0.5220
Epoch 4/200
1000/1000 [==============================] - 0s 303us/step - loss: 0.9526 - acc: 0.4740
Epoch 5/200
1000/1000 [==============================] - 0s 291us/step - loss: 0.9131 - acc: 0.4680
Epoch 6/200
1000/1000 [==============================] - 0s 298us/step - loss: 0.8874 - acc: 0.4720
Epoch 7/200
1000/1000 [==============================] - 0s 301us/step - loss: 0.8212 - acc: 0.4970
Epoch 8/200
1000/1000 [==============================] - 0s 303us/step - loss: 0.8318 - acc: 0.5000
Epoch 9/200
1000/1000 [==============================] - 0s 295us/step - loss: 0.7924 - acc: 0.5090
Epoch 10/200
1000/1000 [==============================] - 0s 308us/step - loss: 0.7869 - a

1000/1000 [==============================] - 0s 279us/step - loss: 0.6816 - acc: 0.5680
Epoch 83/200
1000/1000 [==============================] - 0s 280us/step - loss: 0.6893 - acc: 0.5440
Epoch 84/200
1000/1000 [==============================] - 0s 282us/step - loss: 0.6804 - acc: 0.5630
Epoch 85/200
1000/1000 [==============================] - 0s 282us/step - loss: 0.6802 - acc: 0.5530
Epoch 86/200
1000/1000 [==============================] - 0s 281us/step - loss: 0.6861 - acc: 0.5700
Epoch 87/200
1000/1000 [==============================] - 0s 283us/step - loss: 0.6812 - acc: 0.5640
Epoch 88/200
1000/1000 [==============================] - 0s 280us/step - loss: 0.6891 - acc: 0.5580
Epoch 89/200
1000/1000 [==============================] - 0s 282us/step - loss: 0.6782 - acc: 0.5610
Epoch 90/200
1000/1000 [==============================] - 0s 285us/step - loss: 0.6879 - acc: 0.5630
Epoch 91/200
1000/1000 [==============================] - 0s 289us/step - loss: 0.6822 - acc: 0.5550
Epo

1000/1000 [==============================] - 0s 159us/step - loss: 0.6609 - acc: 0.5980
Epoch 163/200
1000/1000 [==============================] - 0s 156us/step - loss: 0.6613 - acc: 0.5920
Epoch 164/200
1000/1000 [==============================] - 0s 136us/step - loss: 0.6601 - acc: 0.6080
Epoch 165/200
1000/1000 [==============================] - 0s 131us/step - loss: 0.6603 - acc: 0.5970
Epoch 166/200
1000/1000 [==============================] - 0s 141us/step - loss: 0.6636 - acc: 0.5900
Epoch 167/200
1000/1000 [==============================] - 0s 135us/step - loss: 0.6682 - acc: 0.6010
Epoch 168/200
1000/1000 [==============================] - 0s 131us/step - loss: 0.6564 - acc: 0.5990
Epoch 169/200
1000/1000 [==============================] - 0s 146us/step - loss: 0.6599 - acc: 0.6080
Epoch 170/200
1000/1000 [==============================] - 0s 182us/step - loss: 0.6565 - acc: 0.5920
Epoch 171/200
1000/1000 [==============================] - 0s 153us/step - loss: 0.6530 - acc: 0

# Test

In [286]:
final_context_test, final_queries_test= prep_data(final_test_data)

In [287]:
score = model.evaluate([final_queries_test,final_context_test], answers_test, batch_size=128)

1000/1000 [==============================] - 0s 59us/step


In [288]:
print('Final Testing Accuracy: '+ str(score))

Final Testing Accuracy: [0.7255253295898437, 0.5179999995231629]
